In [1]:
LOC = "C:\\Users\\Stephan\\Desktop\\20160606_mg_4_4_2.mdb\\"
FILE = "31deg_long_measure_0906.lsm"

import pims
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import trackpy as tp

# Load image
def load_image(location):
    print("Loading")
    frames = pims.Bioformats(location);
    frames.bundle_axes = ['z', 'y', 'x']
    frames.iter_axes = 't'
    print("Resolution and Frames: ", frames.sizes)
    print("finished loading")
    return frames
    
image = load_image(LOC + FILE)

Loading
Resolution and Frames:  {'y': 1024, 'z': 40, 't': 180, 'x': 1024}
finished loading


In [2]:
class particle_class():

    def __init__(self, Id, xpos, ypos, zpos):
        self.identity = Id
        self.posx = xpos
        self.posy = ypos
        self.posz = zpos
        self.neigbour_list = []

    def get_x(self):
        return self.posx
    def get_y(self):
        return self.posy
    def get_z(self):
        return self.posz

    def add_neigbour(self, neigbour):
        self.neigbour_list.append(neigbour)

    def get_neigbour(self):
        return self.neigbour_list
    
    def get_id(self):
        return self.identity
    

In [ ]:
print("show raw image")
plt.imshow(image[0][0], cmap=plt.cm.gray)
plt.show()

In [ ]:
print("bandpass")
def bandpass(image):  
    '''
    Firt do banpass then normalize.
    '''
    
    raw_frame = image.astype(float)
    
    bandpass = tp.bandpass(raw_frame,lshort=1,llong=10)

    return bandpass

new_image = bandpass(image[179])
#plt.figure()
#plt.imshow(new_image[0], cmap=plt.cm.gray)

#print("show raw image")
#plt.figure()
#plt.imshow(image[0][0], cmap=plt.cm.gray)
#plt.show()

In [ ]:
print("locate")
#features = tp.locate(image[0], diameter=(7, 7, 5), noise_size=2,smoothing_size=(5,5,4),minmass=500)
features = tp.locate(new_image, diameter=(5, 5, 3),minmass=500, noise_size=1)
print("done")

In [ ]:
tp.subpx_bias(features)
plt.show()
print('Features gevonden: {0}'.format(len(features)))

In [ ]:
print("checking if locating went well")
from pylab import rcParams
rcParams['figure.figsize'] = 30, 30
tp.annotate3d(features, new_image)

In [ ]:
plt.figure()
tp.subpx_bias(features)
plt.show()

In [ ]:
plt.figure()
plt.hist(features['mass'], bins = 100)
plt.show()

In [ ]:
print("checking if locating went well")
#from pylab import rcParams
#rcParams['figure.figsize'] = 50, 50
print('found ' + str(len(features)))

tp.annotate3d(features, image[0])

In [ ]:
features.head()

In [ ]:
test = features[features['z']>25]
test

In [ ]:
print("Rezise particles radius")
features['z'] = (features['z'])*0.6
features['x'] = (features['x'] / 1024)*105.77
features['y'] = (features['y'] / 1024)*105.77

In [ ]:
#features_test = features[features['x'] < 10]
#features_test = features[features['z'] < 5]
features_test = features


import math
yas = []
xas = []
zas = []
middle_of_found_x = []
middle_of_found_y = []
middle_of_found_z = []

global cutoff
cutoff = 0.1

print("new list")
for index, row in features_test.iterrows():
    yas.append(row['y'])
    xas.append(row['x'])    #xas[i]=1    #f.ix[i]['x']
    zas.append(row['z'])


max_found = 0
min_found = 99
total = 0.0
print("calculate connectivity")
looprange = len(yas)
print(looprange)

for j in range(looprange):
    if (j % 500 == 1):
        print(j)
    found = 0     
    
    # first find all neigbours
    for i in range(looprange):
        t=math.pow(yas[j]-yas[i],2)+math.pow(xas[j]-xas[i],2)

        # dont add yourself as neigbour
        if(math.sqrt(t) < cutoff and t != 0):
            found += 1
            if(found > max_found):
                max_found = found
            middle_of_found_x.append((xas[j]-xas[i])/2+xas[i])
            middle_of_found_y.append((yas[j]-yas[i])/2+yas[i]) 
            middle_of_found_z.append((zas[j]-zas[i])/2+zas[i])

    if(min_found > found):
        min_found = found
    total += found

print("done")    

In [ ]:
print(max_found)
print(min_found)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
n = 100
xs = features_test['x']
ys = features_test['y']
zs = features_test['z']
ax.scatter(xs, ys, zs, s=200, c='b', marker='.')

#xs = middle_of_found_x
#ys = middle_of_found_y
#zs = middle_of_found_z
#ax.scatter(xs, ys, zs, c='r', marker='x')


ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()


In [ ]:
"""
Connectivity of particles using the particles class

Asumes the particles are seperated by read distance (in micro meters)
"""
#features_test = features[features['x'] < 10]
#features_test = features[features['z'] < 3.4]
features_test = features

import math

global cutoff
cutoff = 7.0 # micro meters

print("Create particles")
particles = []
for index, row in features_test.iterrows():
    newparticle = particle_class(index, row['x'], row['y'], row['z'])
    particles.append(newparticle)

print("calculate connectivity")
print(len(particles))

total = 0
# loop over all particles
for particle in particles:      
    # first find all neigbours
    for neigbour in particles:  
        # dont add yourself as neibour
        if (neigbour.get_id() != particle.get_id()):            
            distance = math.pow(particle.get_x() - neigbour.get_x(),2) + math.pow(particle.get_y() - neigbour.get_y(),2) + math.pow(particle.get_z() - neigbour.get_z(),2)
            if(math.sqrt(distance) < cutoff):
                particle.add_neigbour(neigbour)
                total+=1
print(total)
print("********done************")

In [ ]:
print(min_found)2648
print(max_found)37636
print(total)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
n = 100

xs = []
ys = []
zs = []
for p in particles:
    xs.append(p.get_x())
    ys.append(p.get_y())
    zs.append(p.get_z())
ax.scatter(xs, ys, zs, s=500, c='b', marker='.')

#xs = middle_of_found_x
#ys = middle_of_found_y
#zs = middle_of_found_z

mx = []
my = []
mz = []
for p in particles:
    neigbours = p.get_neigbour()
    for n in neigbours:
        mx.append( ((n.get_x() - p.get_x()) / 2 ) + p.get_x())
        my.append( ((n.get_y() - p.get_y()) / 2 ) + p.get_y())
        mz.append( ((n.get_z() - p.get_z()) / 2 ) + p.get_z())
ax.scatter(mx, my, mz, s=100, c='r', marker='x')


ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
print("done")
plt.show()

In [ ]:
"""
plot neigbours with random colors
warning ovwrite neibours colors
"""

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as colors
import random

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
n = 100

prev = []
for p in particles:
    if p.get_id() not in prev:
        xs = []
        ys = []
        zs = []
        prev.append(p.get_id())

        neigbours = p.get_neigbour()
        xs.append(p.get_x())
        ys.append(p.get_y())
        zs.append(p.get_z())
        for n in neigbours:
            if(n.get_id() not in prev):
                prev.append(n.get_id())
                xs.append( ((n.get_x() - p.get_x()) / 2 ) + p.get_x())
                ys.append( ((n.get_y() - p.get_y()) / 2 ) + p.get_y())
                zs.append( ((n.get_z() - p.get_z()) / 2 ) + p.get_z())
        ax.scatter(xs, ys, zs, s=500, c=random.choice(list(colors.cnames.items()))[1], marker='.')
#random.choice(colors.cnames.keys())

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
print("done")
plt.show()

In [ ]:
'''
plot all clusters
'''
import sys
sys.setrecursionlimit(10**6)

def search(neigbours, xs, ys, zs):
    #print(neigbours)
    if(len(neigbours) > 0):
        for n in neigbours:
            if n.get_id() not in prev:
                xs.append(n.get_x())
                ys.append(n.get_y())
                zs.append(n.get_z())
                prev.append(n.get_id())
                search(n.get_neigbour(), xs, ys, zs)


from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as colors
import random

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
n = 100

prev = []
for p in particles:
    if p.get_id() not in prev:
        xs = []
        ys = []
        zs = []
        prev.append(p.get_id())

        xs.append(p.get_x())
        ys.append(p.get_y())
        zs.append(p.get_z())
        search(p.get_neigbour(), xs, ys, zs)
        ax.scatter(xs, ys, zs, s=500, c=random.choice(list(colors.cnames.items()))[1], marker='.')
#random.choice(colors.cnames.keys())

ax.pbaspect = [1.0, 1.0, 105.77 / 20]
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
print("done")
plt.show()

In [ ]:
'''
Neigbour cahin
'''

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
n = 100
xs = features['x']
ys = features['y']
zs = features['z']
ax.scatter(xs, ys, zs, s=50, c='b', marker='.')
part = particles[1]
neigbours = particle.get_neigbour()
previous = [part.get_id()]
xs = [part.get_x()]
ys = [part.get_y()]
zs = [part.get_z()]
def search(neigbours):
    print(neigbours)
    if(len(neigbours) > 0):
        for n in neigbours:
            if n.get_id() not in previous:
                xs.append(n.get_x())
                ys.append(n.get_y())
                zs.append(n.get_z())
                previous.append(n.get_id())
                search(n.get_neigbour())
search(part.get_neigbour())    
ax.scatter(xs, ys, zs, s=500, c='r', marker='.')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
# make recursion bigger
import sys
sys.setrecursionlimit(10**6)
total = []
for i in range(len(particles)):
    best_route = []
    #print("********************SEARCHING FOR " + str(particles[i].get_id()))
    part = particles[1]
    def search2(this_part, route = []):
        neigbours = this_part.get_neigbour()
        route.append(this_part.get_id())
        for n in neigbours:
            if n.get_id() == particles[i].get_id():
                route.append(n.get_id())
                #print("THE ROUTE: " + str(route))
                #print("found")
                best_route.append(route)
                #return route
            if(n.get_id() not in route): 
                if(len(best_route) > 0):
                    if (len(route) < len(best_route[-1])):
                        search2(n, route = route)
                else:
                    search2(n, route = route)
        #print(route)
        #rint("didnt fount any")
        #print("The best route " + str(best_route))
        return []
    if(len(best_route) > 0):
       print("Beste route found" + str(best_route))
    routen = search2(part)
    total.append(routen)

In [ ]:
particles

In [ ]:
'''
Find connectivity through time
'''